<font size=-1>Licensed under the Apache License, Version 2.0 (the \"License\");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at [https://www.apache.org/licenses/LICENSE-2.0](https://www.apache.org/licenses/LICENSE-2.0)

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an \"AS IS\" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.  See the License for the specific language governing permissions and limitations under the License.</font>


# Inspecting TFX metadata 

In this lab, you explore TFX metadata logged by pipeline runs triggered in lab-02.

An **AI Platform Pipelines** instance includes the [ML Metadata](https://github.com/google/ml-metadata) service. In **AI Platform Pipelines**, ML Metadata uses *MySQL* as a database backend and can be accessed using a GRPC server.

To enable access to the ML Metadata GRPC server, configure Kubernetes port forwarding.

From a JupyterLab terminal, execute the following commands:

```
gcloud container clusters get-credentials [YOUR CLUSTER] --zone [YOURE CLUSTER ZONE]  
kubectl port-forward  service/metadata-service --namespace [YOUR NAMESPACE] 7000:8080
```

In [2]:
!pip install ml-metadata

     |████████████████████████████████| 4.9 MB 7.0 MB/s eta 0:00:01
     |████████████████████████████████| 421.8 MB 29 kB/s s eta 0:00:01    |████▎                           | 56.7 MB 34.7 MB/s eta 0:00:11     |█████████▏                      | 120.1 MB 26.3 MB/s eta 0:00:12     |███████████▌                    | 151.1 MB 50.6 MB/s eta 0:00:06     |████████████████▎               | 213.9 MB 30.2 MB/s eta 0:00:07     |████████████████▎               | 215.1 MB 30.2 MB/s eta 0:00:07     |████████████████████████████▉   | 380.0 MB 49.1 MB/s eta 0:00:01     |█████████████████████████████   | 381.2 MB 49.1 MB/s eta 0:00:01
     |████████████████████████████████| 104 kB 39.0 MB/s eta 0:00:01
     |████████████████████████████████| 3.8 MB 33.5 MB/s eta 0:00:01
     |████████████████████████████████| 63 kB 2.1 MB/s  eta 0:00:01
     |████████████████████████████████| 57 kB 6.1 MB/s  eta 0:00:01
     |████████████████████████████████| 448 kB 40.6 MB/s eta 0:00:01
     |████████████████████████

In [3]:
import os

import ml_metadata

from ml_metadata.metadata_store import metadata_store
from ml_metadata.proto import metadata_store_pb2

In [1]:
import tensorflow_data_validation as tfdv
import tensorflow_model_analysis as tfma




from tfx.orchestration import metadata
from tfx.types import standard_artifacts

ModuleNotFoundError: No module named 'ml_metadata'

## Connecting to ML Metadata 

Configure ML Metadata GRPC client.

In [4]:
grpc_host = 'localhost'
grpc_port = 7000
connection_config = metadata_store_pb2.MetadataStoreClientConfig()
connection_config.host = grpc_host
connection_config.port = grpc_port

Connect to ML Metadata service.

In [5]:
store = metadata_store.MetadataStore(connection_config)

## Exploring ML Metadata 

The Metadata Store uses the following data model:

- `ArtifactType` describes an artifact's type and its properties that are stored in the Metadata Store. These types can be registered on-the-fly with the Metadata Store in code, or they can be loaded in the store from a serialized format. Once a type is registered, its definition is available throughout the lifetime of the store.
- `Artifact` describes a specific instances of an ArtifactType, and its properties that are written to the Metadata Store.
- `ExecutionType` describes a type of component or step in a workflow, and its runtime parameters.
- `Execution` is a record of a component run or a step in an ML workflow and the runtime parameters. An Execution can be thought of as an instance of an ExecutionType. Every time a developer runs an ML pipeline or step, executions are recorded for each step.
- `Event` is a record of the relationship between an Artifact and Executions. When an Execution happens, Events record every Artifact that was used by the Execution, and every Artifact that was produced. These records allow for provenance tracking throughout a workflow. By looking at all Events MLMD knows what Executions happened, what Artifacts were created as a result, and can recurse back from any Artifact to all of its upstream inputs.
- `ContextType` describes a type of conceptual group of Artifacts and Executions in a workflow, and its structural properties. For example: projects, pipeline runs, experiments, owners.
- `Context` is an instances of a ContextType. It captures the shared information within the group. For example: project name, changelist commit id, experiment annotations. It has a user-defined unique name within its ContextType.
- `Attribution` is a record of the relationship between Artifacts and Contexts.
- `Association` is a record of the relationship between Executions and Contexts.

List the registered artifact types.

In [6]:
for artifact_type in store.get_artifact_types():
    print(artifact_type.name)

NoType
Float
Metrics


Display the registered execution types.

In [7]:
for execution_type in store.get_execution_types():
    print(execution_type.name)

DummyExecutionType
components.Bigquery - Query@sha256=4b56389070c6fc5cec38f612fa15bec55a2a8716aff55e8a2350100dcc529b18
components.Submitting a Cloud ML training job as a pipeline step@sha256=6c565ed37269f52c6719afad94a1a44744ed94a0cd54394ba4b26134c4127394
components.Retrieve best run@sha256=f338fae54dd9f9b53afab849686de872350454a8b0eaa215cdf2347d89f16091
components.Evaluate model@sha256=7f8c88edb4a45c3878b6e6f5fc2adbea6f5ccccdeee175ec539a7498a66abaf0
components.Deploying a trained model to Cloud Machine Learning Engine@sha256=0f265efa8481e9ca8ff4d108b2a41220a9e3cdd925b132a3fb5bd13ce80fe38e


List the registered context types.

In [8]:
for context_type in store.get_context_types():
    print(context_type.name)

KfpRun


## Visualizing TFX artifacts

### Retrieve data analysis and validation artifacts

In [9]:
with metadata.Metadata(connection_config) as store:
    stats_artifacts = store.get_artifacts_by_type(standard_artifacts.ExampleStatistics.TYPE_NAME)
    schema_artifacts = store.get_artifacts_by_type(standard_artifacts.Schema.TYPE_NAME)
    anomalies_artifacts = store.get_artifacts_by_type(standard_artifacts.ExampleAnomalies.TYPE_NAME)

NameError: name 'metadata' is not defined

In [ ]:
stats_path = stats_artifacts[-1].uri
train_stats_file = os.path.join(stats_path, 'train', 'stats_tfrecord')
eval_stats_file = os.path.join(stats_path, 'eval', 'stats_tfrecord')
print("Train stats file:{}, Eval stats file:{}".format(
    train_stats_file, eval_stats_file))

schema_file = os.path.join(schema_artifacts[-1].uri, 'schema.pbtxt')
print("Generated schame file:{}".format(schema_file))
anomalies_file = os.path.join(anomalies_artifacts[-1].uri, 'anomalies.pbtxt')
print("Generated anomalies file:{}".format(anomalies_file))

### Visualize statistics

In [ ]:
train_stats = tfdv.load_statistics(train_stats_file)
eval_stats = tfdv.load_statistics(eval_stats_file)
tfdv.visualize_statistics(lhs_statistics=eval_stats, rhs_statistics=train_stats,
                          lhs_name='EVAL_DATASET', rhs_name='TRAIN_DATASET')

### Visualize schema

In [ ]:
schema = tfdv.load_schema_text(schema_file)
tfdv.display_schema(schema=schema)

### Visualize anomalies

In [ ]:
anomalies = tfdv.load_anomalies_text(anomalies_file)
tfdv.display_anomalies(anomalies)

### Retrieve model evaluations

In [ ]:
with metadata.Metadata(connection_config) as store:
    model_eval_artifacts = store.get_artifacts_by_type(standard_artifacts.ModelEvaluation.TYPE_NAME)
    
model_eval_path = model_eval_artifacts[-1].uri
print("Generated model evaluation result:{}".format(model_eval_path))

### Visualize model evaluations

In [ ]:
eval_result = tfma.load_eval_result(model_eval_path)
tfma.view.render_slicing_metrics(
    eval_result, slicing_column='Wilderness_Area')